### IMPORTS

In [1]:
import ipywidgets
import cv2
from IPython.display import display
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
import traitlets
import time
import shutil
import os
import pandas as pd
import numpy as np

<br>

### HYPERPARAMETERS

In [2]:
# 小车控制的油门增益 Offset value for steering control, used to fine-tune the steering behavior of a system.
# Gain value for throttle control, used to adjust the responsiveness or sensitivity of the throttle system.
STEERING_OFFSET = 0.035
THROTTLE_GAIN = 0.7

# 摄像头高度跟宽度 define the width and height of the camera frame
CAMERA_WIDTH = 448
CAMERA_HEIGHT = 336

# 帧率跟是否显示相机提要预览 Frames per second for capturing images
FPS = 10
# Flag to determine whether to show the camera preview or not
SHOW_CAMERA_PREVIEW = False

# 存储数据集的目录路径 Directory path for storing datasets
# TODO: 需換路徑 "datasets/"
DATASETS_DIR = "/home/greg/datasets/"
# 临时数据集的目录路径 Temporary dataset directory path
TMP_DATASET_DIR = DATASETS_DIR + "tmp/"
# 注释文件的文件名  Name of the annotations file
ANNOTATIONS_FILE = "annotations.csv"
# 临时数据集 Path of the temporary annotations file
TMP_ANNOTATIONS = TMP_DATASET_DIR + ANNOTATIONS_FILE

# 数据集的模式 Mode of the dataset
DATASET_MODE = "training"

# Name of the dataset
DATASET_NAME = "3"
# 主数据库目录路径 Main dataset directory path
MAIN_DATASET_DIR = DATASETS_DIR + DATASET_NAME + "_" + DATASET_MODE + "/"
# 主要注释文件的路径 Path of the main annotations file
MAIN_ANNOTATIONS = MAIN_DATASET_DIR + ANNOTATIONS_FILE

<br>

### DATA COLLECTION

In [3]:
#该函数的作用是重置临时数据集目录 Define a function to reset the temporary dataset directory
def reset_temp_dataset_dir():
    #判断临时数据集目录是否存在 Check if the temporary dataset directory exists
    if not os.path.exists(TMP_DATASET_DIR):
        #如果不存在，则使用创建该目录 Create the directory if it doesn't exist
        os.makedirs(TMP_DATASET_DIR)
    else:
        #如果临时数据集目录已经存在，则先删除该目录及其所有内容 If the directory already exists, remove it first 
        shutil.rmtree(TMP_DATASET_DIR)
        #然后再次创建一个空的临时数据集目录 create a new one
        os.makedirs(TMP_DATASET_DIR)

#通过调用reset_temp_dataset_dir()函数，执行重置临时数据集目录 Call the function to reset the temporary dataset directory
reset_temp_dataset_dir()

#判断主数据集目录是否存在。如果主数据集目录不存在，则使用创建该目录 Check if the main dataset directory exists, Create the directory if it doesn't exist
if not os.path.exists(MAIN_DATASET_DIR):
    os.makedirs(MAIN_DATASET_DIR)

In [4]:
# 函数用作于开始录制数据 Define a function to start recording
def start_recording():
    # 用于重置临时数据集目录，以确保准备一个新的录制会话  Reset the temporary dataset directory
    reset_temp_dataset_dir()

   
# 用于保存录制的数据 Define a function to save the recording 
def save_recording():
    # Iterate over files in the temporary dataset directory
    for file in os.listdir(TMP_DATASET_DIR):
        # Check if the file has a '.csv' extension
        if file.endswith('.csv'):
            # 检查主注释文件 MAIN_ANNOTATIONS 是否存在，且不是空的 Check if the main annotations file exists and is not empty
            if os.path.exists(MAIN_ANNOTATIONS) and os.stat(MAIN_ANNOTATIONS).st_size > 0:
                # 打开主注释文件 MAIN_ANNOTATIONS  # Open the main annotations file in append mode
                with open(MAIN_ANNOTATIONS, 'a') as main:
                    # 打开时注释文件 TMP_ANNOTATIONS # Open the temporary annotations file
                    with open(TMP_ANNOTATIONS) as tmp:
                        # 将临时注释文件中的每一行内容追加到主注释文件中 Iterate over lines in the temporary annotations file
                        for line in tmp:
                            main.write(line)
                        # 关闭临时注释文件 Close the temporary annotations file
                        tmp.close()
                    # 关闭主注释文件 Close the main annotations file
                    main.close()
                #跳过将文件移动到主数据集目录
                continue
        #函数将文件从临时数据集目录移动到主数据集目录  Move the file from the temporary dataset directory to the main dataset directory
        shutil.move(TMP_DATASET_DIR+file, MAIN_DATASET_DIR+file)
    #重置临时数据集目录，以准备进行下一次录制 Reset the temporary dataset directory
    reset_temp_dataset_dir()

<br>

### CAR CONTROLLER

In [5]:
# 创建Nvidia Race car 类的实例car  # Create an instance of the NvidiaRacecar class and set throttle_gain and steering_offset attributes
car = NvidiaRacecar()
# 设置car的(油门增益)throttle_gain属性 responsiveness of the throttle control
car.throttle_gain = THROTTLE_GAIN
# 设置car的(方向盘偏移量)steering_offset属性 used to adjust the center or neutral position of the steering control
car.steering_offset = STEERING_OFFSET

In [ ]:
 # 创建控制器小部件  Create a controller widget and display it, It sets up an input controller, which is typically used to control the car remotely
controller = ipywidgets.widgets.Controller(index=0)
# 显示控制器小部件 It allows the user to see and interact with the controller interface
display(controller)
# 暂停1秒, 可改变暂时时间 Pause the execution for 1 second
time.sleep(1.0)

In [9]:
# 定义clamp function用于将值限制在给定的范围内 Define a function to clamp a value within a given range
def clamp(value, val_min=-1.0, val_max=1.0):
    return min(val_max, max(val_min, value))

# 定义is_valid_press function，用于判断游戏手柄按钮event是否为有效的按下的event # Define a function to check if a button press event is valid
def is_valid_press(x):
    return x['name'] == 'pressed' and x['new']

# Specific to Gamepad Mode 2 # 特定于游戏手柄模式2  

# 函数建立了与游戏手柄的连接，并创建控制器的方向盘轴值与car的转向属性之间的关联 
# Link the value of the controller's axis to the car's steering, applying the clamp function to the value
steering_link = traitlets.dlink((controller.axes[2], 'value'), (car, 'steering'), transform=lambda x: clamp(-x))
# 函数建立了与游戏手柄的连接，并创建控制器的油门轴值与car的油门属性之间的关联
# Link the value of the controller's axis to the car's throttle, applying the clamp function to the value
throttle_link = traitlets.dlink((controller.axes[1], 'value'), (car, 'throttle'), transform=lambda x: clamp(x))

# 函数建立了与游戏手柄的连接，并创建控制器的刹车按钮值与car的油门属性之间的关联
#  Link the value of the controller's button to the car's throttle, setting it to 0.0
brake_throttle_link = traitlets.dlink((controller.buttons[7], 'value'), (car, 'throttle'), transform=lambda x: 0.0)
#函数建立了与游戏手柄的连接，并创建控制器的半油门按钮值与car的油门属性之间的关联
# Link the value of the controller's button to the car's throttle, setting it to -0.5 if the button value is greater than 0.5, otherwise set it to 0
half_throttle_link = traitlets.dlink((controller.buttons[5], 'value'), (car, 'throttle'), transform=lambda x: -0.5 if x > 0.5 else 0)

# 当控制器上的特定按钮被按下时调用start_recording() function开始录制
# Observe the press of the controller's button 2 and call the start_recording function if the press event is valid
controller.buttons[2].observe(lambda x: start_recording() if is_valid_press(x) else ())
# 当控制器上的特定按钮被按下时调用save_recording() function开始保存录制内容
# Observe the press of the controller's button 1 and call the save_recording function if the press event is valid
controller.buttons[1].observe(lambda x: save_recording() if is_valid_press(x) else ())

<br>

### CAMERA

In [10]:
# 创建了一个 CSICamera 类的实例 camera，用于获取摄像头图像
# Create an instance of the CSICamera class with specified width, height, and capture_fps, It represents the camera used to capture frames
camera = CSICamera(width=CAMERA_WIDTH, height=CAMERA_HEIGHT, capture_fps=FPS)
# 启动摄像头 Set the camera's running attribute to True to start capturing frames
camera.running = True

In [11]:
# 定义函数以保存带有注释的相机帧 Define a function to save an annotated camera frame
def save_annotated_camera_frame(frame):
    # Generate a timestamp based on the current time
    timestamp = str(int(time.time()*1000))
    # Encode the frame as a JPEG image
    encoded_image = cv2.imencode('.jpg', frame)[1]
    
    # 保存相机帧的函数 Define a function to save the camera frame
    def save_camera_frame():
        cv2.imwrite(TMP_DATASET_DIR+timestamp+".jpg", frame)

    # 保存时间戳、汽车的转向和油门值以特定的格式的函数     Define a function to save the annotation
    def save_annotation():
        with open(TMP_ANNOTATIONS, 'a') as f: 
            f.write(timestamp + ", " + str(round(car.steering, 2)) + ", " + str(round(car.throttle, 2)) + "\n")
            f.close()
    
    # 调用保存相机帧到临时数据集目录 Call the functions to save the camera frame and annotation
    save_camera_frame()
    # 调用保存注释到临时注释文件
    save_annotation()
    
    # 将帧图像编码为 JPEG 格式的图像 Re-encode the frame as a JPEG image
    encoded_image = cv2.imencode('.jpg',frame)[1]
    # 将图像数据转换为字节类型然后返回 Return the encoded image as bytes
    return bytes(encoded_image)

In [12]:
# 创建一个 ipywidgets.Image 对象，并设置图像的格式为 JPEG # Create an instance of the ipywidgets Image class with JPEG format
image = ipywidgets.Image(format='jpeg')
# 如果调用到SHOW_CAMERA_PREVIEW 就会将图像显示出来 If SHOW_CAMERA_PREVIEW is True, display the image widget
if SHOW_CAMERA_PREVIEW:
    display(image)

In [13]:
# 创建相机和图像之间的关联，并通过save_annotated_camera_frame函数进行转换和保存
# Link the camera's value attribute (frames) to the image widget's value attribute, applying the save_annotated_camera_frame function as a transform 
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=save_annotated_camera_frame)